In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import load_iris
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklvq import GMLVQ
from timeit import timeit
from sklearn.model_selection import KFold

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import (
    cross_val_score,
    RepeatedKFold,
)


matplotlib.rc("xtick", labelsize="small")
matplotlib.rc("ytick", labelsize="small")

In [2]:
def getdata():
    data = pd.read_csv("feature_vectors.csv").to_numpy()
    labels = pd.read_csv("diagnosis_label.csv").to_numpy().squeeze()
    labelscenter = pd.read_csv("center_label.csv").to_numpy().squeeze()

    labelsfinal = labels + labelscenter
    
    # better way to perform this
    labelsdiseases1 = np.where(labelsfinal == 'HCUMCG') 
    centerlabels1 = labelsfinal[labelsdiseases1]
    labelsdiseases2 = np.where(labelsfinal == 'HCUGOSM') 
    #labelsdiseases = np.where(labels == 'HCCUN','1',centerlabels) 
    centerlabels2 = labelsfinal[labelsdiseases2]
    labelsdiseases3 = np.where(labelsfinal == 'HCCUN') 
    centerlabels3 = labelsfinal[labelsdiseases3]
    
    # better way to perform this
    centerlabels = np.concatenate((centerlabels1,centerlabels2,centerlabels3))
    centerdata = data[labelsdiseases1]
    centerdata = np.concatenate((centerdata,data[labelsdiseases2]))
    centerdata = np.concatenate((centerdata,data[labelsdiseases3]))
    
    
    data = ztransform(data)
    centerdata = ztransform(centerdata)
    return data,labels,centerdata,centerlabels

In [3]:
def ztransform(data_ztransform):
    # Sklearn's standardscaler to perform z-transform
    scaler = StandardScaler()

    # Compute (fit) and apply (transform) z-transform
    # why do we do an inverse z transform
    data_ztransform = scaler.fit_transform(data_ztransform)
    
    return data_ztransform

In [4]:
def model_definition_center():
    
    model = GMLVQ(
    distance_type="adaptive-squared-euclidean",
    activation_type="sigmoid",
    activation_params={"beta": 2},
    solver_type="sgd",
    solver_params={"max_runs": 20,"batch_size":1,"step_size": np.array([0.1, 0.05])},
    random_state=1428,)
    
    return model

In [5]:
def model_definition_disease(correctionmatrix):
    model1 = GMLVQ(
        distance_type="adaptive-squared-euclidean",
        activation_type="sigmoid",
        activation_params={"beta": 2},
        solver_type="sgd",
        solver_params={"max_runs": 20,"batch_size":1,"step_size": np.array([0.1, 0.05])},
        random_state=1428,
        relevance_correction=correctionmatrix
    )
    return model1

In [6]:
def train_modelkfold(data, label,disease,correctionmatrix,folds=10):
    modelmatrix = np.zeros((folds,folds),dtype=object) 
    accuracies = np.zeros((folds,folds),dtype=float) 
    transformed_model_matrix = np.zeros((folds,folds),dtype=object) 
    transformed_data_matrix = np.zeros((folds,folds),dtype=int)
    
    for repeated in range(2):
        print("========Repeated fold number",str(repeated),"========")
        kfold = KFold(folds, shuffle=True)
        for k, (training_indices, testing_indices) in enumerate(kfold.split(data)):
            if disease==False:
                model = model_definition_center()
                print('in this')
                print(type(model))
            else:
                model = model_definition_disease(correctionmatrix)
            accuracy = 0
            correct = 0
            trainX, trainY, testX, testY = data[training_indices], label[training_indices], data[testing_indices], label[testing_indices]
            print(np.shape(trainX))
            model.fit(trainX, trainY)
            #validation accuracy
            # could use model.predict but it will only give the fraction of the correctly predicted labels
            #_, accuraccy = model.evaluate(testX, testY)
            predictedlabels = model.predict(testX)

            # since no model.evaluate.....

            for i in range(len(predictedlabels)):
                if(predictedlabels[i]==testY[i]):
                    correct = correct+1

            accuracy = correct/len(testY)
            print('%.3f accuracy' % (correct/len(testY)))
            
            # stored row wise
            modelmatrix[repeated,k] = model
        #histories.append(history_kfold)
    return modelmatrix

In [7]:
def calculate_lambda(modelmatrix):
    #average_lambda = np.mean(sum(model.lambda_ for model in modellist),axis=0)
    #check the shape of total_lambda
    
    total_lambda = np.zeros((36,36),dtype = 'float')
    
    for i in range(2):
        
        lambdaperfold = np.zeros((36,36),dtype='float')        
        
        for modelin in modelmatrix[i,:]:
            lambdaperfold = lambdaperfold + modelin.lambda_
            
        # addition per fold divided by total number of models in a fold
        lambdaperfold = lambdaperfold/np.shape(modelmatrix)[1]
        
        total_lambda = total_lambda + lambdaperfold
        #total lambda = average lambda, avergae lambda is divided by total number of repeats
        
    total_lambda = total_lambda/2
    # check if this assignment is necessary                     
    #average_lambda = total_lambda/np.shape(modelmatrix)[0]
    return total_lambda

In [8]:
def eigendecomposition(average_lambda):
    eigenvalues, eigenvectors = np.linalg.eigh(average_lambda)
    # Flip (reverse the order to descending) before assigning.
    eigenvalues = np.flip(eigenvalues)

    # eigenvectors are column matrix in ascending order. Flip the columns and transpose the matrix
    # to get the descending ordered row matrix.
    eigenvectors = np.flip(eigenvectors, axis=1).T
    
    return eigenvalues,eigenvectors

In [9]:
feature_names = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z','ab','ac','ad','ae','af','ag','ah','ai','aj','ak']


In [10]:

def correction_matrix(eigenvectors):
    N=36
    K=2
    I = np.identity(N)
    outerproduct = np.zeros((N,N))   
    for i in range(K):
        outerproduct = np.outer(eigenvectors[i,:].T,eigenvectors[i,:])
    correctionmatrix = I-outerproduct
    return correctionmatrix

In [11]:
# check if this is the right label for disease
data,labels,centerdata,centerlabels = getdata()

centermodel = train_modelkfold(centerdata,centerlabels,False,None)

average_lambda_center = calculate_lambda(centermodel)

print(np.sum(np.diagonal(average_lambda_center)))

eigenvalues,eigenvectors = eigendecomposition(average_lambda_center)


correctionmatrix = correction_matrix(eigenvectors)

diseasemodel = train_modelkfold(data,labels,True,correctionmatrix)

correction_matrix(eigenvectors)

average_lambda_disease = calculate_lambda(diseasemodel)


print(np.sum(np.diagonal(average_lambda_disease)))


========Repeated fold number 0 ========
in this
<class 'sklvq.models._gmlvq.GMLVQ'>
(73, 36)
0.889 accuracy
in this
<class 'sklvq.models._gmlvq.GMLVQ'>
(73, 36)
0.889 accuracy
in this
<class 'sklvq.models._gmlvq.GMLVQ'>
(74, 36)
1.000 accuracy
in this
<class 'sklvq.models._gmlvq.GMLVQ'>
(74, 36)
0.875 accuracy
in this
<class 'sklvq.models._gmlvq.GMLVQ'>
(74, 36)
1.000 accuracy
in this
<class 'sklvq.models._gmlvq.GMLVQ'>
(74, 36)
0.875 accuracy
in this
<class 'sklvq.models._gmlvq.GMLVQ'>
(74, 36)
0.750 accuracy
in this
<class 'sklvq.models._gmlvq.GMLVQ'>
(74, 36)
1.000 accuracy
in this
<class 'sklvq.models._gmlvq.GMLVQ'>
(74, 36)
1.000 accuracy
in this
<class 'sklvq.models._gmlvq.GMLVQ'>
(74, 36)
0.750 accuracy
========Repeated fold number 1 ========
in this
<class 'sklvq.models._gmlvq.GMLVQ'>
(73, 36)
1.000 accuracy
in this
<class 'sklvq.models._gmlvq.GMLVQ'>
(73, 36)
0.667 accuracy
in this
<class 'sklvq.models._gmlvq.GMLVQ'>
(74, 36)
1.000 accuracy
in this
<class 'sklvq.models._gmlvq.

In [13]:
print(np.sum(np.diagonal(average_lambda_center)))

1.0000000000000002


In [13]:
total_lambda = np.zeros((36,36),dtype=object)
for model in centermodel[1,:]:
    print((model.lambda_))

[[ 1.44372422e-02 -6.46018724e-03 -2.53069819e-03 ...  4.28875495e-04
   4.85006228e-02  3.76203061e-04]
 [-6.46018724e-03  3.31198612e-03 -4.89364678e-04 ...  8.60992979e-04
  -2.17648523e-02 -1.30940910e-04]
 [-2.53069819e-03 -4.89364678e-04  7.77727639e-03 ... -4.81493851e-03
  -8.98192961e-03 -2.59109416e-04]
 ...
 [ 4.28875495e-04  8.60992979e-04 -4.81493851e-03 ...  3.09601297e-03
   1.75272917e-03  1.33436477e-04]
 [ 4.85006228e-02 -2.17648523e-02 -8.98192961e-03 ...  1.75272917e-03
   1.63740494e-01  1.29271522e-03]
 [ 3.76203061e-04 -1.30940910e-04 -2.59109416e-04 ...  1.33436477e-04
   1.29271522e-03  1.70074674e-05]]
[[ 0.00500893  0.00331451 -0.00516531 ...  0.00329795  0.02055103
  -0.00109537]
 [ 0.00331451  0.02252399 -0.00865626 ...  0.01136391 -0.02114551
  -0.00070319]
 [-0.00516531 -0.00865626  0.00669894 ... -0.0057733  -0.01228066
   0.00112403]
 ...
 [ 0.00329795  0.01136391 -0.0057733  ...  0.00632285 -0.00213635
  -0.00071214]
 [ 0.02055103 -0.02114551 -0.012280

In [20]:
a = np.zeros(36,dtype=object)

In [21]:
a

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=object)

In [ ]:
np.sum(np.diagonal(average_lambda_disease))

In [8]:
diesease_model = train_modelkfold(centerdata,centerlabels)
calculate k fold for the correction matrix
average_lambda_disease = calculate_lambda()

In [12]:
np.sum(np.diagonal(average_lambda))
# average model.lambda aka kfold relevance for disease after correction
# average model.lambda aka kfold relevance for disease without correction
# both above without k fold

1.0

# Eigen value decompostion of the average lambda to derive eigen values and eigen vectors

In [ ]:
len(eigenvalues)

In [ ]:
def ploteigenvalues(eigenvalues,eigenvectors):
   # Plot the eigenvalues of the eigenvectors of the relevance matrix.

    %matplotlib inline 
    fig, ax = plt.subplots()
    fig.suptitle("Eigenvalues")
    ax.bar(range(0, len(eigenvalues)), eigenvalues)
    ax.set_ylabel("Weight")
    ax.grid(False)
    plt.savefig('centereigenvalues.png')


    # Plot the first two eigenvectors of the relevance matrix, which  is called `omega_hat`.
    fig, ax = plt.subplots()
    fig.suptitle("First Eigenvector")
    ax.bar(feature_names, eigenvectors[0, :])
    ax.set_ylabel("Weight")
    ax.grid(False)
    plt.savefig('Firstvectorcenter.png')

    
    fig, ax = plt.subplots()
    fig.suptitle("Second Eigenvector")
    ax.bar(feature_names, eigenvectors[1, :])
    ax.set_ylabel("Weight")
    ax.grid(False)
    plt.savefig('Secondvectorcenter.png')

    fig, ax = plt.subplots()
    # The relevance matrix is available after fitting the model.
    relevance_matrix = average_lambda
    fig.suptitle("Relevance Matrix Diagonal Center data")
    ax.bar(feature_names, np.diagonal(relevance_matrix))
    ax.set_ylabel("Weight")
    ax.grid(False)
    plt.savefig('centerrelevance.png')


In [ ]:
ploteigenvalues(eigenvalues,eigenvectors)

In [ ]:
len(correctionmatrix)

In [ ]:

# Train the model using the scaled data and true labels
model1.fit(data, labels)

# Predict the labels using the trained model
predicted_labels = model1.predict(data)
# To get a sense of the training performance we could print the classification report.
#print(classification_report(labelssubclass, predicted_labels))

In [ ]:
# trained omega
omegad = model1.omega_

In [ ]:
# Plot the eigenvalues of the eigenvectors of the relevance matrix.
fig, ax = plt.subplots()
fig.suptitle("Eigenvalues")
ax.bar(range(0, len(model1.eigenvalues_)), model1.eigenvalues_)
ax.set_ylabel("Weight")
ax.grid(False)
plt.savefig('diseaseeigenvalues.png')


# Plot the first two eigenvectors of the relevance matrix, which  is called `omega_hat`.
fig, ax = plt.subplots()
fig.suptitle("First Eigenvector")
ax.bar(feature_names, model1.omega_hat_[0, :])
ax.set_ylabel("Weight")
ax.grid(False)
plt.savefig('Firstvectordisease.png')


fig, ax = plt.subplots()
fig.suptitle("Second Eigenvector")
ax.bar(feature_names, model1.omega_hat_[1, :])
ax.set_ylabel("Weight")
ax.grid(False)
plt.savefig('Secondvectordisease.png')


fig, ax = plt.subplots()
# The relevance matrix is available after fitting the model.
relevance_matrix = model1.lambda_
fig.suptitle("Relevance Matrix Diagonal Center data")
ax.bar(feature_names, np.diagonal(relevance_matrix))
ax.set_ylabel("Weight")
ax.grid(False)
plt.savefig('disease123.png')

In [ ]:
# removing the z transform
data = scaler.inverse_transform(data)
transformed_data = model1.transform(data, scale=True)

x_d = transformed_data[:, 0]
y_d = transformed_data[:, 1]

# Transform the model, i.e., the prototypes (scaled by square root of eigenvalues "scale = True")
# prototype inverser transform
prototypes = scaler.inverse_transform(model1.prototypes_)

transformed_model1 = model1.transform(prototypes, scale=True)

print(len(model1.prototypes_))
x_m = transformed_model1[:, 0]
y_m = transformed_model1[:, 1]

fig, ax = plt.subplots()
fig.suptitle("Maindataset disease data with corresponding prototypes")
colors = ['yellow','Magenta','brown']#,'pink','lightgreen']
for i, cls in enumerate(model1.classes_):
    ii = cls == labels
    ax.scatter(
        x_d[ii],
        y_d[ii],
        c=colors[i],
        s=100,
        alpha=0.7,
        edgecolors="white",
        label=model1.classes_[model1.prototypes_labels_[i]],
    )
ax.scatter(x_m, y_m, c=colors, s=180, alpha=0.8, edgecolors="black", linewidth=2.0)
ax.set_xlabel("First eigenvector")
ax.set_ylabel("Second eigenvector")
ax.legend()
ax.grid(True)
print(model1.classes_)
plt.savefig('disease.png')
#plt.savefig('destination_path23.eps', format='eps')
print('A1: Black, A2: Red, B1: Green, B2: Blue, Centers A and B, and diseases 1 and 2.) ')

In [ ]:
print(modellist[1])